In [2]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s

print()

In [3]:
def second(seq):
    return seq[1]


def zero_sum_sub_array(a: List[int]) -> Tuple[int, int]:
    "Return the indexes of the longest sub-array of `a` which sums to zero."

    # The idea is to compute an array of the accumulated values.
    # If any accumulated value shows up in the array again, there is a zero sub-array.
    # Store the accumulated values to prevent recomputation below.
    acc = list(accumulate(a))

    # Determine the first occurrence of the sum.
    # Special care is given for an empty array with 0 having -1 as index.
    occ = {0: -1}
    for i, e in enumerate(acc):
        occ.setdefault(e, i)
    # Determine the longest span between this occurrence and the first one.
    length, j = min(starmap(lambda j, e: (occ[e] - j, j), enumerate(acc)))
    return j + 1 + length, j + 1


def zero_sum_sub_matrix(m: List[List[int]]) -> List[List[int]]:
    "Return the largest sub-matrix of `m` that sums to zero."
    # The idea is borrowed from the 2D Kadane's algorithm.
    # We use top, bottom cutoff to accumulate the values for the columns into an array.
    # Then we apply the `zero_sum_sub_array` above to determine the left and right brackets.
    # Using top, bottom, left, and right we calculate the area,
    # which is used to determine the largest matrix.
    r, c = len(m), len(m[0])
    mx = (0, 0, 0, 0, 0)
    for top in range(r):
        a = [0] * c
        for bottom in range(top, r):
            mb = m[bottom]
            for k in range(c):
                a[k] += mb[k]
            left, right = zero_sum_sub_array(a)
            area = (bottom + 1 - top) * (right - left)
            x = (-area, left, right - left, top, bottom + 1)
            mx = min(mx, x)

    # `top` and `left` are inclusive, `bottom` and `right` are exclusive here.
    (area, left, width, top, bottom) = mx
    return [r[left : left + width] for r in m[top:bottom]] if area else []

In [4]:
zero_sum_sub_matrix([[1, 2, 3], [-3, -2, -1], [1, 7, 5]])

[[1, 2, 3], [-3, -2, -1]]

In [246]:
zero_sum_sub_matrix([[9, 7, 16, 5], [1, -6, -7, 3], [1, 8, 7, 9], [7, -2, 0, 10]])

[[-6, -7], [8, 7], [-2, 0]]

In [258]:
m = matrix.make(
    """\
-4 6 6
1 7 8
0 -11 -6
4 5 -1
10 -6 -7
-9 -7 0
-7 1 -2
-9 7 -2
-6 -2 -8
5 5 0
1 0 6
3 1 6
-2 -6 -7
-9 4 5
7 -1 -3
-2 -7 -11
8 -11 -3
-5 -4 10
1 4 7"""
)
pprint(m)

[[-4, 6, 6],
 [1, 7, 8],
 [0, -11, -6],
 [4, 5, -1],
 [10, -6, -7],
 [-9, -7, 0],
 [-7, 1, -2],
 [-9, 7, -2],
 [-6, -2, -8],
 [5, 5, 0],
 [1, 0, 6],
 [3, 1, 6],
 [-2, -6, -7],
 [-9, 4, 5],
 [7, -1, -3],
 [-2, -7, -11],
 [8, -11, -3],
 [-5, -4, 10],
 [1, 4, 7]]


In [278]:
zero_sum_sub_matrix(m)

[[-6, -2], [5, 5], [1, 0], [3, 1], [-2, -6], [-9, 4], [7, -1]]

In [265]:
m2 = matrix.make(
    """\
-6 -2
5 5
1 0
3 1
-2 -6
-9 4
7 -1"""
)
pprint(m2)

[[-6, -2], [5, 5], [1, 0], [3, 1], [-2, -6], [-9, 4], [7, -1]]


In [266]:
sum(sum(r) for r in m2)

0

In [268]:
min((-14, 1, 2, 10, 20), (-14, 0, 2, 10, 20))

(-14, 0, 2, 10, 20)